<a href="https://colab.research.google.com/gist/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e/untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music genre classification notebook

## Importing Libraries

In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
C:\Users\anchora\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\anchora\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\anchora\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\anchora\Anaconda3\lib\site-packages\tensorflow\python\framework\d

## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

## Extracting the Spectrogram for every Audio

In [3]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

<Figure size 720x720 with 0 Axes>

All the audio files get converted into their respective spectrograms .WE can noe easily extract features from them.

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [4]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [7]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

The data has been extracted into a [data.csv](https://github.com/parulnith/Music-Genre-Classification-with-Python/blob/master/data.csv) file.

# Analysing the Data in Pandas

In [8]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,42.351029,...,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues,NaN
1,blues.00001.wav,0.340983,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,35.874684,...,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues,NaN
2,blues.00002.wav,0.363603,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,31.689014,...,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues,NaN
3,blues.00003.wav,0.404779,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,26.871927,...,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues,NaN
4,blues.00004.wav,0.308590,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,22.153301,...,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues,NaN


In [9]:
data.shape

(1000, 28)

In [10]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [102]:
genre_list = data.iloc[:, -2]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [103]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-2], dtype = float))

## Dividing data into training and Testing set

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [105]:
len(y_train)

800

In [106]:
len(y_test)

200

In [107]:
X_train[10]

array([ 1.0809865 ,  1.82811392,  2.02956805,  1.91588982,  0.80382962,
        0.18933106, -1.54412456,  2.18758715, -1.30886149,  1.42241602,
       -0.84451773,  1.75447541, -0.85562714,  1.05323963,  0.25725675,
        0.46946067, -0.35597318,  0.91674961, -0.45582964,  1.07006469,
       -0.43926629,  0.97883847,  0.9367992 ,  1.08682651,  2.43267917])

In [108]:
print(np.shape(y_train))

(800,)


# Classification with Keras

## Building our Network

In [110]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [111]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [112]:

history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                   

Epoch 1/20
800/800 [==============================] - 1s 689us/step - loss: 2.1923 - accuracy: 0.2013
Epoch 2/20
800/800 [==============================] - 0s 50us/step - loss: 1.8965 - accuracy: 0.3625
Epoch 3/20
800/800 [==============================] - 0s 50us/step - loss: 1.6798 - accuracy: 0.4150
Epoch 4/20
800/800 [==============================] - 0s 50us/step - loss: 1.5010 - accuracy: 0.4837
Epoch 5/20
800/800 [==============================] - 0s 50us/step - loss: 1.3387 - accuracy: 0.5650
Epoch 6/20
800/800 [==============================] - 0s 40us/step - loss: 1.2311 - accuracy: 0.5825
Epoch 7/20
800/800 [==============================] - 0s 40us/step - loss: 1.1306 - accuracy: 0.6212
Epoch 8/20
800/800 [==============================] - 0s 50us/step - loss: 1.0515 - accuracy: 0.6500
Epoch 9/20
800/800 [==============================] - 0s 50us/step - loss: 0.9876 - accuracy: 0.6787
Epoch 10/20
800/800 [==============================] - 0s 50us/step - loss: 0.9349 - accur

In [113]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 1ms/step


In [114]:
print('test_acc: ',test_acc)

test_acc:  0.6200000047683716


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [115]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 20 epochs:

In [116]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 1s 895us/step - loss: 2.2797 - accuracy: 0.1383 - val_loss: 2.1295 - val_accuracy: 0.3650
Epoch 2/30
600/600 [==============================] - 0s 70us/step - loss: 2.1007 - accuracy: 0.3400 - val_loss: 2.0191 - val_accuracy: 0.3750
Epoch 3/30
600/600 [==============================] - 0s 73us/step - loss: 1.9762 - accuracy: 0.3467 - val_loss: 1.9266 - val_accuracy: 0.3800
Epoch 4/30
600/600 [==============================] - 0s 60us/step - loss: 1.8680 - accuracy: 0.3617 - val_loss: 1.8254 - val_accuracy: 0.3900
Epoch 5/30
600/600 [==============================] - 0s 60us/step - loss: 1.7598 - accuracy: 0.3833 - val_loss: 1.7119 - val_accuracy: 0.4000
Epoch 6/30
600/600 [==============================] - 0s 60us/step - loss: 1.6538 - accuracy: 0.4183 - val_loss: 1.6210 - val_accuracy: 0.4250
Epoch 7/30
600/600 [==============================] - 0s 80us/step - loss: 1.5568 - accuracy: 0

In [117]:
results

[1.3226342964172364, 0.5350000262260437]

## Predictions on Test Data

In [118]:
predictions = model.predict(X_test)

In [119]:
predictions[0].shape

(10,)

In [120]:
np.sum(predictions[0])

1.0

In [121]:
np.argmax(predictions[0])

8